# Transferencia de Calor y Masa <br> *Trabajo Práctico: Intercambiador de calor de casco y tubos*

## Juan Cruz Gonzalez Sembla
### 1er Cuatrimestre de 2023

### INTERCAMBIADORES DE CALOR


#### OBJETIVO

Diseño de un intercambiador de tipo casco y tubos para el calentamiento de etilenglicol, procesado en una industria química. 

#### ENUNCIADO


Se requieren calentar una cantidad $\dot m_F$ de etilenglicol desde un temperatura $T_{FE}$, garantizando a la salida una temperatura  $T_{FS}=40^\circ$ C. Para ello se dispone de un caudal $\dot m_C$ de agua que ingresa a $T_{CE}=65^\circ$C. El agua no podrá enfriarse por debajo de los $T_{CS}=20^\circ$ C por requerimientos del sistema de calentamiento de la planta. A los efectos del proceso, no afecta que bajo situaciones especiales la temperatura del etilenglicol esté por encima de lo establecido. Considerar al agua como el fluido que pasa por los tubos y al etilenglicol en un por la carcasa.

<br>

#### Se pide:

   1. Determinar las características de un intercambiador de casco y tubos que cumpla con los requerimientos. 
   2. Calcular las caídas de presión de ambas corrientes en su paso por el equipo.
   3. Completar el cuadro de resumen que contenga las características constructivas y de operación fundamentales calculadas (Entregar el archivo Excel completo adjunto).
   4. Hacer un plano conceptual a escala del equipo.
   5. Hacer un plano a escala de la placa portatubos y su relación con la carcasa.
   6. Durante la operación, el proceso sufre cierto desbalance que hace que la temperatura de entrada del etilenglicol disminuya 5ºC. Manteniendo los caudales constantes, determinar las nuevas temperaturas de salida. ¿Verifican las condiciones de funcionamiento requeridas? (Se recomienda utilizar el método e-NUT).
   7. En los casos que no verifiquen, se dispone de un sistema de control del caudal de agua, para poder llevar al equipo dentro de los parámetros de funcionamiento adecuados. ¿Cuál es este nuevo caudal? (Se recomienda utilizar el método e-NUT)

## Inicialización de librerías

<br>

Lo primero que debemos hacer antes de escribir el código, es llamar a las librerías (o *packages*) que vamos a usar a lo largo del trabajo práctico. Utilizaremos librerías básicas como `numpy` y `matplotlib`, así como también librerías especializadas como `ht`, `CoolProp` y `fluids`.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import ht
import CoolProp as cp
import fluids

<br>

## Paso 1: Datos del problema


| Parámetro      | Valor |
| :---        |    :----:   |
| Corriente de Servicio:      | Agua de caldera tratada       |
| Corriente de Proceso:   | Etilenglicol        |
| Temperatura de etilenglicol a la entrada - $T_{FE}$ [°C] | 5 |
| Temperatura de etilenglicol a la salida - $T_{FS}$ [°C] | 40 |
| Temperatura del agua a la entrada - $T_{CE}$ [°C] | 65 |
| Temperatura mínima del agua a la entrada - $T_{CSmin}$ [°C] | 20|
| Caudal de etilenglicol - $\dot{m}_F$ [kg/s] | 4.5 |

Ahora podemos inicializar las variables:

In [2]:
Tfe = 5  # °C
Tfs = 40  # °C
Tce = 65  # °C
Tcs_min = 20  # °C
m_f = 4.5  # kg/s

## Paso 1: Datos del problema

Calcularemos los parámetros de funcionamiento del intercambiador a través de las ecuaciones de conservación de energía:

\begin{equation}
\tag{1}
\dot{Q} = \dot{m}_F \cdot C_F \cdot (T_{FS} - T_{FE})
\end{equation}

\begin{equation}
\tag{2}
\dot{Q} = \dot{m}_C \cdot C_C \cdot (T_{CS} - T_{CE})
\end{equation}

<br>

## Paso 1: Datos del problema


Además, contamos con la expresión que permite calcular el calor en función  del coeficiente global de transmisión $U$, el aérea de intercambio $A$, del diferencial de temperatura logarítmica media en cotra corriente $DTLM_{cc}$ y un factor de corrección de la diferencia de temperaturas $F_t$:

\begin{equation}
\tag{3}
\dot{Q} = UA \cdot DTLM_{cc} \cdot F_t
\end{equation}

donde el diferencial de temperatura logarítmica media en contra corriente puede calcularse de la siguiente manera:

\begin{equation}
DTLM_{cc} = \frac{(T_{cs} - T_{fe}) - (T_{ce} - T_{fs})}{\text{ln} \Big( \frac{T_{cs} - T_{fe}}{T_{ce} - T_{fs}} \Big) }
\end{equation}

Es visible que podemos calcular el calor a partir de (1). Sin embargo, tenemos dos incognitas ($\dot{m}_c$ y $T_{cs}$) que nos impiden seguir adelante. Para resolver este problema, debemos adelantarnos a los pasos 2 y 3.


## Paso 2: Pasos por carcasa

<br>

Como criterio de diseño tomaremos un intercambiador de un solo paso por carcasa $Np_s$. Luego, podemos inicializar la respectiva variable:


In [3]:
Nps = 1

<br>

## Paso 3: Obtener $F_t$

En la referencia "Intercambiadores de calor" de E. Cao, podemos ver que no se aconseja diseñar con factores de corrección $F_t$ menores a 0,75 ya que el factor se torna excesivamente sensible a cambios en los parámetros, disminuyendo de manera drástica. El valor del coeficiente $F_t$ puede ser calculado de la siguiente manera:

\begin{equation}
\tag{4}
F_t = \frac{\sqrt{R^2 + 1}}{R - 1} \frac{\text{ln}\frac{1 - Px}{1 - R P_x}}{\text{ln} \frac{\frac{2}{P_x} - 1 - R + \sqrt{R^2 + 1}}{\frac{2}{P_x} - 1 - R - \sqrt{R^2 + 1}}}
\end{equation}

donde los parámetros $S$, $R$ y $P_x$ están definidos como sigue:

\begin{equation}
S = \frac{T_{fs} - T_{fe}}{T_{ce} - T_{fe}} \qquad \qquad \qquad R = \frac{T_{ce} - T_{cs}}{T_{fs} - T_{fe}} \qquad \qquad \qquad P_x = \frac{1 - \Big( \frac{RS - 1}{S - 1} \Big)^\frac{1}{Np_s}}{R - \Big( \frac{RS - 1}{S - 1} \Big)^\frac{1}{Np_s}}
\end{equation}

<br>
<br>

Ahora vamos a hacer un solver que nos permita calcular la $T_{cs}$ mínima de manera que se cumpla $F_t > 0,75$

In [4]:
#Inicializamos las variables
Tcs = Tcs_min
Ft = 0

while Ft <= 0.75:
    #Calculamos R y S
    S = (Tfs - Tfe) / (Tce - Tfe)
    R = (Tce - Tcs) / (Tfs - Tfe)
    
    #Ahora debemos pedir que R sea distinto de 1 para evitar división por cero
    if R != 1:
        Px = (1 - ((R * S - 1) / (S - 1)) ** (1 / Nps)) / (R - ((R * S - 1) / (S - 1)) ** (1 / Nps))
        
        # Luego creamos variables auxiliares A, B y C para facilitar el cálculo.
        A = np.sqrt(R ** 2 + 1) / (R - 1)
        B = (1 - Px) / (1 - R * Px)
        C = 2 / Px - 1 - R
        D = (C + np.sqrt(R ** 2 + 1)) / (C - np.sqrt(R ** 2 + 1))

        # Si B > 0 y D > 0 podemos calcular los logaritmos
        if B > 0 and D > 0:
            Ft = A * np.log(B) / np.log(D)

    Tcs += 0.001  # Agregamos un dT a nuestra variable de interés
    
print('La temperatura de salida del agua es Tcs = %.2f °C'%Tcs)

La temperatura de salida del agua es Tcs = 37.25 °C


Tomaremos un margen de seguridad de 5°C:

In [5]:
Tcs = np.ceil(Tcs + 5)

print('La temperatura de salida del agua es Tcs = %.2f °C'%Tcs)

La temperatura de salida del agua es Tcs = 43.00 °C


Luego, podemos regresar al paso 1.

## Paso 1: Datos del problema

Vamos a calcularlas propiedades de los fluidos a la temperatura promedio entre la entrada y la salida:

In [6]:
Tfm = (Tfe + Tfs) / 2
Tcm = (Tce + Tcs) / 2

print('La temperatura media del etilenglicol es Tfm = %.2f °C'%Tfm)
print('La temperatura media del agua es Tcm = %.2f °C'%Tcm)

La temperatura media del etilenglicol es Tfm = 22.50 °C
La temperatura media del agua es Tcm = 54.00 °C


## Paso 1: Datos del problema

Luego, podemos usar la librería `CoolProp` para obtener las propiedades. Para el agua es directo, solo debemos indicar la presión de trabajo (que será atmosférica) y la temperatura media que calculamos previamente:

In [7]:
C_c = cp.CoolProp.PropsSI('C', 'T', Tcm + 273, 'P', 101325, 'Water')  # Calor específico [J/kgK]
mu_c = cp.CoolProp.PropsSI('V', 'T', Tcm + 273, 'P', 101325, 'Water')  # Viscosidad [kg/ms]
Pr_c = cp.CoolProp.PropsSI('PRANDTL', 'T', Tcm + 273, 'P', 101325, 'Water')  # Prandtl
k_c = cp.CoolProp.PropsSI('L', 'T', Tcm + 273, 'P', 101325, 'Water')  # Conductividad [W/mK]
rho_c = cp.CoolProp.PropsSI('D', 'T', Tcm + 273, 'P', 101325, 'Water')  # Densidad [kg/m^3]


print('El calor especifico del agua a la temperatura media C_c = %.2f J/kgK'%C_c)
print('La viscosidad del agua a la temperatura media mu_c = %.2e kg/ms'%mu_c)
print('El número de Prandtl del agua a la temperatura media Pr_c = %.2f'%Pr_c)
print('La conductividad del agua a la temperatura media k_c = %.2f W/mK'%k_c)
print('La densidad del agua a la temperatura media rho_c = %.2f kg/m^3'%rho_c)

El calor especifico del agua a la temperatura media C_c = 4182.55 J/kgK
La viscosidad del agua a la temperatura media mu_c = 5.13e-04 kg/ms
El número de Prandtl del agua a la temperatura media Pr_c = 3.33
La conductividad del agua a la temperatura media k_c = 0.64 W/mK
La densidad del agua a la temperatura media rho_c = 986.25 kg/m^3


## Paso 1: Datos del problema

Sin embargo, para el etilenglicol debemos tener en cuenta que el mismo es utilizado disuelto en agua, y la concentración de etylenglicol será tanto mayor cuanto menor sea el punto de congelamiento requerido por el sistema. En la siguiente figura tomada de la [referencia](https://www.ladco.com.ar/determinacion-de-etilenglicol-o-propilenglicol-en-el-fluido-refrigerante-ya-cargado-en-un-chiller-o-heladera/) podemos ver la variación del punto de congelamiento en función de la concentración:

<img src="images/water_ethylene_glycol_freezing-points.png" width=40% style="margin-left:auto; margin-right:auto">

## Paso 1: Datos del problema

Luego, para evitar problemas de congelamiento en nuestro circuito, usaremos una solución de 20% en agua. En el paquete de `CoolProp` la solución de etilenglicol en agua forma parte de la lista de fluidos incompresibles que podemos encontrar en la [documentación](https://coolprop.zegil.com.br/fluid_properties/Incompressibles.html) del paquete. Calculamos entonces las propiedades:

In [8]:
C_f = cp.CoolProp.PropsSI('C', 'T', Tfm + 273, 'P', 101325, 'INCOMP::MEG-20%')  # Calor específico [J/kgK]
mu_f = cp.CoolProp.PropsSI('V', 'T', Tfm + 273, 'P', 101325, 'INCOMP::MEG-20%')  # Viscosidad [kg/ms]
Pr_f = cp.CoolProp.PropsSI('PRANDTL', 'T', Tfm + 273, 'P', 101325, 'INCOMP::MEG-20%')  # Prandtl
k_f = cp.CoolProp.PropsSI('L', 'T', Tfm + 273, 'P', 101325, 'INCOMP::MEG-20%')  # Conductividad [W/mK]
rho_f = cp.CoolProp.PropsSI('D', 'T', Tfm + 273, 'P', 101325, 'INCOMP::MEG-20%')  # Densidad [kg/m^3]

print('El calor especifico del etilenglicol a la temperatura media C_f = %.2f J/kgK'%C_f)
print('La viscosidad del etilenglicol a la temperatura media mu_f = %.2e kg/ms'%mu_f)
print('El número de Prandtl del etilenglicol a la temperatura media Pr_f = %.2f'%Pr_f)
print('La conductividad del etilenglicol a la temperatura media k_f = %.2f W/mK'%k_f)
print('La densidad del etilenglicol a la temperatura media rho_f = %.2f kg/m^3'%rho_f)

El calor especifico del etilenglicol a la temperatura media C_f = 3900.45 J/kgK
La viscosidad del etilenglicol a la temperatura media mu_f = 1.56e-03 kg/ms
El número de Prandtl del etilenglicol a la temperatura media Pr_f = 11.90
La conductividad del etilenglicol a la temperatura media k_f = 0.51 W/mK
La densidad del etilenglicol a la temperatura media rho_f = 1023.34 kg/m^3


<br>

## Paso 1: Datos del problema

Luego, podemos calcular el calor usando (1):

In [9]:
Q = m_f * C_f * (Tfs - Tfe)

print('El calor intercambiado es Q = %.2f W'%Q)

El calor intercambiado es Q = 614321.55 W


Así, podemos despejar el caudal másico con (2):

In [10]:
m_c = Q / (C_c * (Tce - Tcs))  # kg/s

print('El caudal másico del agua es m_c = %.2f kg/s'%m_c)

El caudal másico del agua es m_c = 6.68 kg/s


Ahora, para el paso 2 ya sabemos que $Np_s = 1$, por lo que pasamos al cálculo de $F_t$, es decir, el paso 3. Recordamos que cuando obtuvimos $T_{cs}$, no usamos la temperatura que resultó del solver sino una levemente superior, razón por la cual debemos recalcular el $F_t$.

## Paso 3: Obtener $F_t$

Para obtener $F_t$ usaremos una de las funcionalidades internas del paquete `ht`.

In [11]:
Ft = ht.F_LMTD_Fakheri(Tci=Tfe, Tco=Tfs, Thi=Tce, Tho=Tcs, shells=1)

print('El valor de Ft = %.3f'%Ft)

El valor de Ft = 0.847


## Paso 4: Adoptar U

Ahora adoptaremos un valor estimado para el coeficiente global de transmisión $U_{est}$. Tomaremos el valor medio sugerido por la referencia bibliográfica "Intercambiadores de calor" de E. Cao (pág. 153) para el intercambio entre agua y solventes orgánicos:

In [12]:
U = 560  # W/m^2K

## Paso 5: 

Calcularemos el área estimada a través de la ecuación (3), para lo cual debemos primero computar el valor del diferencial de temperatura logarítmica media. Para el cálculo del $DTLM_{cc}$ haremos uso de otra de las funcionalidades internas de la librería `ht`:

In [13]:
DTLM_cc = ht.LMTD(Tci=Tfe, Tco=Tfs, Thi=Tce, Tho=Tcs)  # °C

print('El valor del diferencial de temperatura logarítmica media es DTLM_cc = %.2f °C'%DTLM_cc)

El valor del diferencial de temperatura logarítmica media es DTLM_cc = 31.05 °C


Luego, podemos calcular el área estimada de intercambio $A_{est}$:

In [14]:
A = Q / (U * DTLM_cc * Ft)  # m^2

print('El área de intercambio estimada es A = %.2f m^2'%A)

El área de intercambio estimada es A = 41.72 m^2


## Paso 6: Selección de tubos

Tomaremos tubos de 1 pulgada de diámetro y de espesor 8 según norma BWG, de longitud 3 m. En la referencia bibliográfica "Intercambiadores de calor" de E. Cao (pág. 139) tenemos un cuadro que indica las dimensiones de los tubos según esta norma:

<img src="images/dimensiones-tubos-norma-BWG.png" width=20% style="margin-left:auto; margin-right:auto">

## Paso 6

Luego, podemos inicializar las variables correspondientes a los tubos seleccionados:

In [15]:
D = 0.0254  # m
Di = 0.017  # m
e = (D - Di) / 2  # m 
Lt = 3  # m

## Paso 7: Cálculo de la cantidad de tubos $N_t^{est}$

La cantidad de tubos multiplicada por la superficie exterior del tubo es igual al área de intercambio estimada en el paso 5:

\begin{equation}
\tag{5}
N_t = \frac{A_{est}}{\pi D L_t}
\end{equation}

In [16]:
Nt = int(np.ceil(A / (np.pi * D * Lt)))

print(f'La cantidad de tubos es Nt = {Nt}')

La cantidad de tubos es Nt = 175


## Paso 8: Cálculo del número de pasos por tubos $Np_t$

Para tener altos coeficientes de intercambio pero con la precaución de que el $\Delta P$ no aumente de manera considerable, buscamos un número de Reynolds en los tubos superior a 10000:

\begin{equation}
Re_d = \frac{4 \dot{m}_T}{\pi D_i \mu} \frac{Np_t}{N_t^{est}} > 10000 \Rightarrow Np_t > \frac{10^4 N_t^{est} \pi \mu D_i}{4 \dot{m}_T}
\end{equation}

In [17]:
Npt = 10**4 * Nt * np.pi * mu_c * Di / (4 * m_c)
Npt = int(np.ceil(Npt / 2) * 2)  # Buscamos el próximo par superior

print(f'La cantidad de pasos por tubos es Npt = {Npt}')

La cantidad de pasos por tubos es Npt = 2


## Paso 9: Tamaño de carcasa $D_s$

Haremos uso de una placa portatubos de cabezales fijos con una distribución de tubos en cuadrado. El pitch $P_t$ recomendado es de 1,25 veces el diámetro exterior, tal y como podemos ver en la referencia "Intercambiadores de calor" de E. Cao. Para seleccionar el diámetro de la carcasa $D_s$, podemos usar la tabla que encontramos en la página 140 del libro, o usar una de las [funciones internas](https://github.com/CalebBell/ht/blob/master/ht/hx.py
) del paquete `ht`:

In [18]:
Pt = 1.25 * D  # m

Ds = ht.DBundle_for_Ntubes_Phadkeb(Nt, D, Pt, Npt, angle=90)  # m

print("El diámetro de carcasa es Ds = %.2f''"%(Ds / 0.0254))

El diámetro de carcasa es Ds = 20.53''


## Paso 9: Tamaño de carcasa $D_s$
Sin embargo, como puede ocurrir que el diámetro resultante no coincida con los disponibles, hemos automatizado la selección del diámetro de carcasa por medio de la creación de un diccionario que contiene todos los diámetros de la tabla. El proceso consiste en buscar el diametro inmediatamente superior. Luego, recalculamos el número de tubos que se corresponde con este diámetro de carcasa $D_s$, esta vez usando otra de las funcionalidades de `ht`:

In [19]:
dict_Ds = {'Diametros':[8, 10, 12, 13.25, 15.25, 17.25, 19.25, 21.25, 23.25, 25, 27, 29, 31, 33, 35, 37]}

idx = (np.abs((Ds / 0.0254) * np.ones(np.shape(dict_Ds['Diametros'])[0])  - dict_Ds['Diametros'])).argmin()

if (Ds / 0.0254) < dict_Ds['Diametros'][idx]:
    Ds = dict_Ds['Diametros'][idx] * 0.0254  # m
else: 
    Ds = dict_Ds['Diametros'][idx + 1]  * 0.0254  # m

Nt = ht.Ntubes(Ds, D, Pt, Ntp=Npt, angle=90, Method=None)
# Nt = ht.Ntubes_Phadkeb(Ds, D, Pt, Ntp=Npt, angle=90)

print("El diámetro de carcasa es Ds = %.2f''"%(Ds / 0.0254))
print(f'El número de tubos es Nt = {Nt}')

El diámetro de carcasa es Ds = 21.25''
El número de tubos es Nt = 196


## Paso 10: Cálculo de la superficie de intercambio real $A_{real}$

Como hemos modificado la cantidad de tubos, debemos volver a calcular el área real:

\begin{equation}
A_{real} = \pi D L_t N_t
\end{equation}

In [20]:
A = np.pi * Lt * D * Nt

print('El área de intercambio real es A = %.2f m^2'%A)

El área de intercambio real es A = 46.92 m^2


## Paso 11: Verificar el $U_{est}$

Podemos calcular el $U$ de la siguiente manera:

\begin{equation}
\tag{6}
U = \frac{1}{\frac{1}{U^{limpio}} + R_{fi} \frac{D}{D_i} + R_{fe}}
\end{equation}

donde $R_{fi}$ y $R_{fe}$ son las resistencias de ensuciamiento en el área interior y exterior respectivamente. 

## Paso 11: Verificar el $U_{est}$

El coeficiente global de transmición $U^{limpio}$ se obtiene según:

\begin{equation}
U^{limpio} = \frac{1}{\frac{D}{h_i D_i} + \frac{\text{ln}(D/D_i)D}{2 k_w} + \frac{1}{h_e}}
\end{equation}

donde podemos ver en el denominador la suma de un término asociado a la convección del lado interior, la conducción a través de la pared del tubo y la convección del lado exterior. Como el término conductivo es despreciable respecto a la convección en interna y externa, podemos escribir $U^{limpio}$ de la siguiente manera:

\begin{equation}
\tag{7}
U^{limpio} = \frac{1}{\frac{D}{h_i D_i} + \frac{1}{h_e}}
\end{equation}

Luego, podemos ver que para aplicar las ecuaciones (6) y (7) debemos obtener los coeficientes de intercambio $h_i$ y $h_e$.

## Paso 12: Cálculo del coeficiente de convección interior $h_i'$

El coeficiente de convección interno sin corregir puede calcularse de la siguiente manera:

\begin{equation}
\tag{8}
h_i' = \frac{Nu_t' k_c}{D_i}
\end{equation}

donde $Nu_t'$ es el Nusselt de los tubos sin corregir. Para calcular este parámetro, podemos hacer uso de la ecuación (4.4) de la referencia "Intercambiadores de calor" de E. Cao (pág. 71), donde para flujo turbulento (i.e. $Re_d > 10000$) tenemos:

\begin{equation}
\tag{9}
Nu_t' = 0.023 Re_d^{0.8} Pr^{1/3}
\end{equation}

## Paso 12: Cálculo del coeficiente de convección interior $h_i'$


Por otra parte, también podemos usar una de las funciones internas de `ht`, previo cálculo del nuevo número de $Re_d$ (ya que el número de tubos ha sido modificado en el paso 9).

In [21]:
Re_d = 4 * m_c* Npt / (np.pi * mu_c * Di * Nt)

Nu_c = ht.conv_internal.turbulent_Colburn(Re_d, Pr_c)
print("El número de Nusselt interno sin corregir es Nu_c = %.2f"%Nu_c)

El número de Nusselt interno sin corregir es Nu_c = 54.19


Luego podemos calcular el coeficiente de intercambio interno sin corregir usando la ecuación (8):

In [22]:
h_c = Nu_c * k_c / Di  # W/K

print("El coeficiente de intercambio interno sin corregir es h_c = %.2f W/K"%h_c)

El coeficiente de intercambio interno sin corregir es h_c = 2055.33 W/K


<br>

## Paso 13: Cálculo del área de flujo del lado de la carcasa

Como podemos ver en la referencia "Intercambiadores de calor" de E. Cao (pág. 75), dada la dificultad de definir una velocidad del flujo por carcasa, Kern propone el cálculo de un área que se corresponde con la hilera hipotética de tubos que pasa por la carcasa:

<div>
    <div style="display: inline-block;  width: 50%;">
Luego, podemos escribir:

\begin{equation}
\tag{10}
A_s = \frac{c B D_s}{P_t}
\end{equation}

donde c es el claro entre tubos (que puede calcularse como $c = P_t - D$) y B es la separación entre bafles que consideramos como $0.25 D_s$.
    </div>
    <div style="display: inline-block; vertical-align: text-bottom; width: 40%;">
        <br>
        <img src="images/area-flujo-carcasa.png" width=40% style="margin-left:auto; margin-right:auto">
    </div>
</div>



In [23]:
As = (Pt - D) * 0.25 * Ds * Ds / Pt  # m^2 - Recordamos que Pt = 1.25 D --> c = Pt - D = 1.25 D - D = 0.25 D

print("El área del flujo del lado carcasa es As = %.2f m^2"%As)

El área del flujo del lado carcasa es As = 0.01 m^2


## Paso 14: Cálculo del diámetro equivalente $D_e$

La definición del diámetro equivalente de Kern para el cálculo del número de Reynolds es la siguiente:

\begin{equation}
\tag{11}
D_e = \frac{4 P_t^2 - \pi D^2}{\pi D}
\end{equation}

In [24]:
De = (4 * Pt ** 2 - np.pi * D **2) / (np.pi * D)  # m

print("El diámetro equivalente es De = %.3f m"%De)

El diámetro equivalente es De = 0.025 m


## Paso 15: Cálculo del número de Reynolds por carcasa $Re_s$

Con el área de flujo del lado de la carcasa y el diámetro equivalente podemos obtener $Re_s$:

\begin{equation}
\tag{12}
Re_s = \frac{D_e \dot{m}_s}{A_s \mu}
\end{equation}

In [25]:
Re_s = De * m_f / (mu_f * As)

print("El número de Reynols del lado carcasa es Re_s = %.3f"%Re_s)

El número de Reynols del lado carcasa es Re_s = 4986.253


## Paso 16: Cálculo del coeficiente de convección exterior $h_e'$

Para el cálculo de este coeficiente haremos uso de la ecuación (5.7) que encontramos en la referencia "Intercambiadores de calor" de E. Cao  (pág. 77):

\begin{equation}
\tag{13}
Nu_s' = 0.36 Re_s^{0.55} Pr^{1/3}
\end{equation}

In [26]:
Nu_f = 0.36 * Re_s ** 0.55 * Pr_f ** (1 / 3)

print("El número de Nusselt externo sin corregir es Nu_f = %.2f"%Nu_f)

El número de Nusselt externo sin corregir es Nu_f = 88.84


Luego, el coeficiente de convección exterior sin corregir será:

In [27]:
h_f = Nu_f * k_f / D  # W/K

print("El coeficiente de intercambio externo sin corregir es h_f = %.2f W/K"%h_f)

El coeficiente de intercambio externo sin corregir es h_f = 1784.83 W/K


## Paso 17: Cálculo de la temperatura promedio de la pared $T_w$ y corrección de los coeficientes

La temperatura promedio de la pared puede ser calculada de la siguiente manera:

\begin{equation}
\tag{14}
\bar{T}_w = \frac{h_i' \bar{T}_i + h_e' \bar{T}_e}{h_i' + h_e'}
\end{equation}

In [28]:
Tw = (h_c * Tcm + h_f * Tfm) / (h_c + h_f)  # °C

print("La temperatura promedio de la pared es Tw = %.2f °C"%Tw)

La temperatura promedio de la pared es Tw = 39.36 °C


## Paso 17: Cálculo de la temperatura promedio de la pared $T_w$ y corrección de los coeficientes

Luego, podemos corregir los coeficientes de la siguiente manera:

\begin{equation}
h_i = h_i' \Big( \frac{\mu}{\mu_w} \Big)^{0.14} \qquad \qquad h_e = h_e' \Big( \frac{\mu}{\mu_w} \Big)^{0.14}
\end{equation}

Para lo cual debemos calcular las viscosidades de los fluidos en la pared.

In [29]:
mu_fw = cp.CoolProp.PropsSI('V', 'T', Tw + 273, 'P', 101325, 'INCOMP::MEG-20%')  # Viscosidad [kg/ms]
mu_cw = cp.CoolProp.PropsSI('V', 'T', Tw + 273, 'P', 101325, 'Water')  # Viscosidad [kg/ms]

h_c = h_c * (mu_c / mu_cw) ** 0.14
h_f = h_f * (mu_f / mu_fw) ** 0.14

print("El coeficiente de intercambio interno corregido es h_c = %.2f W/K"%h_c)
print("El coeficiente de intercambio externo corregido es h_f = %.2f W/K"%h_f)

El coeficiente de intercambio interno corregido es h_c = 1983.01 W/K
El coeficiente de intercambio externo corregido es h_f = 1890.95 W/K


## Paso 18: Cálculo del U

Ahora aplicamos las ecuaciones (7) y luego (6) para el cálculo de $U^{limpio}$ y $U$ respectivamente. La referencia bibliográfica "Intercambiadores de calor" de E. Cao (pág. 155) recomienda los riguientes valores para las resistencias de ensuciamiento para fluidos industriales:

\begin{equation}
R_{fi} = R_{fe} = 0.0002 \frac{m^2 K}{W}
\end{equation}

In [30]:
Ulimpio = 1 / (D / (Di * h_c) + 1 / h_f)   # W/m^2K

Rf = 2e-4  # m^2K/W
U = 1 / (1 / Ulimpio + Rf * (1 + D / Di))  # W/m^2K

print("El coeficiente global de transmisión es U = %.2f W/m^2K"%U)

El coeficiente global de transmisión es U = 561.45 W/m^2K


## Paso 19: Verificación de la superficie de intercambio

Debemos calcular el área $A_{calc}$ según la ecuación (3):

\begin{equation}
A_{calc} = \frac{\dot{Q}}{U DTLM_{cc} F_t}
\end{equation}

In [31]:
Acalc = Q / (U * DTLM_cc * Ft)  # m^2

print("El área de cálculo es Acalc = %.2f m^2"%Acalc)
print("El área real de intercambio es A = %.2f m^2"%A)

El área de cálculo es Acalc = 41.61 m^2
El área real de intercambio es A = 46.92 m^2


Aqui debemos buscar que se cumpla $Acalc < Areal$. En caso de no cumplirse, debemos volver a correr el código, modificando el valor de U que seleccionamos al inicio para que sea similar al obtenido en el paso 18. Lo ideal es que el área real sea entre un 10 y un 20% mayor a la calculada.

<br>

## Paso 20: Cálculo de la caída de presión - Lado tubos

La pérdida de carga será igual a la suma de la pérdida en los tubos más la que se produce en los cabezales por el cambio de dirección. Las expresiones y explicaciones pueden ser encontradas en la referencia bibliográfica "Intercambiadores de calor" de E. Cao (pág. 72).

\begin{equation}
\tag{15}
\Delta P_{tubos} = \Delta P_t + \Delta P_r
\end{equation}

donde,

\begin{equation}
\Delta P_t = 4 f Np_t \frac{L}{D_i} \Big( \frac{\dot{m}_t}{A_c} \Big)^2 \frac{1}{2 \rho} \Big( \frac{\mu}{\mu_w} \Big)^a \qquad \Delta P_r = 4 Np_t \Big( \frac{\dot{m}_t}{A_c} \Big)^2 \frac{1}{2 \rho}
\end{equation}

siendo $a = -0.14$ en régimen turbulento y $f$ el factor de fricción, que puede ser calculado de la siguiente manera para un flujo turbulento:

\begin{equation}
f = 1.2 \Big( 0.0014 + \frac{0.125}{Re_d^{0.32}} \Big)
\end{equation}

## Paso 20: Cálculo de la caída de presión - Lado tubos

Pasamos a los cálculos:

In [32]:
ft = 1.2 * (0.0014 + 0.125 / Re_d ** .32)

Gt = m_c * Npt * 4 / (Nt * np.pi * Di **2)  # kg/m^2s

DeltaP_t = 4 * ft * Npt * Lt * Gt ** 2 / (2 * Di * rho_c) * (mu_c / mu_cw) ** -0.14  # Pa

DeltaP_r = 4 * Npt * Gt ** 2 / rho_c / 2  # Pa

DeltaP_tubos = DeltaP_t + DeltaP_r

print("La pérdida de carga en los tubos es DeltaP_t = %.2f Pa"%DeltaP_t)
print("La pérdida de carga en los cabezales es DeltaP_r = %.2f Pa"%DeltaP_r)
print("La pérdida de carga total en los tubos es DeltaP_tubos = %.2f Pa"%DeltaP_tubos)

La pérdida de carga en los tubos es DeltaP_t = 639.23 Pa
La pérdida de carga en los cabezales es DeltaP_r = 365.35 Pa
La pérdida de carga total en los tubos es DeltaP_tubos = 1004.58 Pa


<br>

## Paso 21: Cálculo de la caída de presión - Lado carcasa

La pérdida de carga en la carcasa puede ser calculada, como podemos ver en la referencia bibliográfica "Intercambiadores de calor" de E. Cao (pág. 77), de la siguiente manera:

\begin{equation}
\tag{16}
\Delta P_{carcasa} = f (N_b + 1) \frac{D_s}{D_e} \Big( \frac{\dot{m}_s}{A_s} \Big)^2 \frac{1}{2 \rho} \Big( \frac{\mu}{\mu_w} \Big)^{0.14}
\end{equation}

donde $N_b$ es el número de bafles y $N_b + 1$ es el número de veces que el fluído cruza el haz de tubos. El factor de fricción puede calcularse según:

\begin{equation}
f = \text{e} ^ {5.1858\ -\ 1.7645\ \text{ln}(Re_s)\ +\ 0.13357\ \text{ln} (Re_s) ^2}
\end{equation}

In [33]:
Nb = np.floor(Lt / (0.25 * Ds))

fc = np.exp(5.1858 - 1.7645 * np.log(Re_s) + 0.13357 * np.log(Re_s) ** 2)

DeltaP_carcasa = fc * (Nb + 1) *  Ds / De * (m_f / As) ** 2 * 1 / (2 * rho_f) * (mu_f / mu_fw) ** 0.14  # Pa

print("La pérdida de carga en la carcasa es DeltaP_carcasa = %.2f Pa"%DeltaP_carcasa)

La pérdida de carga en la carcasa es DeltaP_carcasa = 20906.05 Pa


## Paso 23: Desempeño del equipo

Haremos uso del método $\epsilon - NUT$ para evaluar el comportamiento del equipo ante una modificación en los parámetros. En una primera instancia, consideramos una caída en la temperatura de entrada del etilenglicol de 5°C. Sabemos que, visto que la eficiencia $\varepsilon$ puede ser calculada como función de dos parámetros que son independientes de la temperatura ($\varepsilon = \varepsilon(C_R, NUT))$, ésta permanecerá constante ante la perturbación en la temperatura de entrada del etilenglicol. Por otra parte, también podemos calcular la eficienciencia a partir de su definición:

\begin{equation}
\tag{17}
\varepsilon = \frac{\Delta T_{max}}{T_{ce} - T_{fe}}
\end{equation}

donde el $\Delta T_{max}$ es aquel sufrido por el fluído con menor capacidad térmica. Veamos entonces si se trata del etilenglicol o del agua y calculemos la eficiencia para la condición de diseño.

## Paso 23: Desempeño del equipo

In [34]:
if C_c * m_c < C_f * m_f:
    print("El fluído caliente tiene la menor capacidad térmica")
    Cmin = C_c * m_c
    Cmax = C_f * m_f
else:
    print("El fluído frío tiene la menor capacidad térmica")
    Cmin = C_f * m_f
    Cmax = C_c * m_c

epsilon = (Tfs - Tfe) / (Tce - Tfe)

print("El valor de la eficiencia es epsilon = %.3f"%epsilon)

El fluído frío tiene la menor capacidad térmica
El valor de la eficiencia es epsilon = 0.583


## Paso 23: Desempeño del equipo

Ahora, considerando que la eficiencia permanece constante, podemos despejar la $T_{fs}$ para la condición perturbada, es decir, $T_{fe} = 0°C$:

\begin{equation}
T_{fs} = \varepsilon (T_{ce} - T_{fe}) + T_{fe}
\end{equation}

In [35]:
Tfe_old = Tfe  # °C - Guardamos el valor de la temperatura de entrada de diseño

Tfe = 0  # °C

Tfs = epsilon * (Tce - Tfe) + Tfe  # °C

print("La temperatura de salida del etilenglicol es Tfs = %.2f °C"%Tfs)

La temperatura de salida del etilenglicol es Tfs = 37.92 °C


## Paso 23: Desempeño del equipo

Luego, podemos calcular el calor intercambiado:

In [36]:
Q_old = Q

print("El calor intercambiado previo a la perturbación es Q_old = %.2f W"%Q_old)

Q = m_f * C_f * (Tfs - Tfe)  # W

print("El nuevo calor intercambiado es Q = %.2f W"%Q)

El calor intercambiado previo a la perturbación es Q_old = 614321.55 W
El nuevo calor intercambiado es Q = 665515.01 W


## Paso 23: Desempeño del equipo

Y la temperatura de salida del agua:

In [37]:
Tcs_old = Tcs  # °C - Guardamos el valor de la temperatura de salida de diseño

Tcs = Tce - Q / (C_c * m_c)  # °C

print("La temperatura de salida del agua es Tcs = %.2f °C"%Tcs)

La temperatura de salida del agua es Tcs = 41.17 °C


Luego, podemos ver que no alcanzamos la condición de diseño, ya que la temperatura de salida del etilenglicol $T_{fs}$ no alcanza los 40°C.

## Paso 23: Desempeño del equipo

Ahora, debemos calcular el caudal de agua que debemos aplicar para lograr que $T_{fs}$ sea nuevamente 40°C. Para ello, comenzamos por calcuar la eficiencia para esta nueva condición:

In [38]:
Tfs = 40  # °C

epsilon = (Tfs - Tfe) / (Tce - Tfe)

print("El valor de la nuvea eficiencia es epsilon = %.3f"%epsilon)

El valor de la nuvea eficiencia es epsilon = 0.615


Luego, podemos calcular el valor del número de unidades de transferencia:

In [39]:
NUT = U * A / Cmin

print("El número de unidades de transferencia es NUT = %.3f"%NUT)

El número de unidades de transferencia es NUT = 1.501


<br>

## Paso 23: Desempeño del equipo

Ahora, debemos calcular el valor de $C_R$ de la definición de eficiencia:

\begin{equation}
\tag{18}
\varepsilon = 2 \Bigg( 1 + C_R + \sqrt{1 + C_R^2} \frac{1 + \text{e}^{-NUT\sqrt{1 + C_R^2}}}{1 - \text{e}^{-NUT\sqrt{1 + C_R^2}}} \Bigg) ^ {-1}
\end{equation}

Para ello, vamos a calcular de manera iterativa hasta encontrar el valor de $C_R$ que otorgue la eficiencia que calculamos en la slide anterior. 



In [40]:
Cr = 0
epsilon_it = 0

while abs(epsilon - epsilon_it) > 1e-5:
    A = np.sqrt(1 + Cr ** 2)
    B = -NUT * A
    epsilon_it = 2 * (1 + Cr + A * (1 + np.exp(B)) / (1 - np.exp(B))) ** (-1)
    Cr += 0.0001
    
print("El resultado obtenido es Cr = %.4f"%Cr)

El resultado obtenido es Cr = 0.5949


## Paso 23: Desempeño del equipo

Luego, de la definición de $C_R$ podemos despejar $\dot{m}_c$:

\begin{equation}
C_R = \frac{(\dot{m} C)_{min}}{(\dot{m} C)_{max}} \quad \Rightarrow \quad \dot{m}_c = \frac{\dot{m}_f C_f}{C_R C_c}
\end{equation}

In [41]:
m_c_old = m_c

m_c = m_f * C_f / (Cr * C_c)

print("El nuevo caudal de agua deberá ser m_c = %.2f kg/s"%m_c)
print("Se requiere un aumento en el caudal de agua del %.2f %%"%((m_c - m_c_old) / m_c_old * 100))

El nuevo caudal de agua deberá ser m_c = 7.05 kg/s
Se requiere un aumento en el caudal de agua del 5.66 %


## Paso 23: Desempeño del equipo

Finalmente, podemos calcular el calor intercambiado y la temperatura de salida del agua:

In [42]:
Q = m_f * C_f * (Tfs - Tfe)  # W

print("El nuevo calor intercambiado es Q = %.2f W"%Q)

Tcs = Tce - Q / (C_c * m_c)  # °C

print("La temperatura de salida del agua es Tcs = %.2f °C"%Tcs)

El nuevo calor intercambiado es Q = 702081.77 W
La temperatura de salida del agua es Tcs = 41.20 °C


## Paso 24: Exportación de datos

Por último, exportaremos los datos de diseño:

In [44]:
import xlwt as xw
import pandas as pd
# Workbook is created 
wb = xw.Workbook() 
xw.add_palette_colour("gris", 0x21)
wb.set_colour_RGB(0x21, 200, 200,200)  
# add_sheet is used to create sheet. 
sheet1 = wb.add_sheet('Performance Unidad') 
style = xw.easyxf('pattern: pattern solid, fore_colour gris;')
# Input data into rows 
sheet1.write(3, 0, 'Nombre Fluido') 
sheet1.write(3, 1, 'Agua') 
sheet1.write(3, 2, 'Etilenglicol') 

sheet1.write(4, 0, 'Caudal masico (kg/s)',style) 
sheet1.write(4, 1, '%.2f'%(m_c_old), style) 
sheet1.write(4, 2, '%.2f'%(m_f),style) 

sheet1.write(5, 0, 'Temperatura Entrada') 
sheet1.write(5, 1, '%.1f'%(Tce)) 
sheet1.write(5, 2, '%.1f'%(Tfe_old)) 

sheet1.write(6, 0, 'Temperatura salida',style) 
sheet1.write(6, 1, '%.1f'%(Tcs_old),style) 
sheet1.write(6, 2, '%.1f'%(Tfs),style) 

sheet1.write(7, 0, 'Densidad kg/m3') 
sheet1.write(7, 1, '%.2f'%rho_c) 
sheet1.write(7, 2, '%.2f'%rho_f) 

sheet1.write(8, 0, 'Viscosidad cP',style) 
sheet1.write(8, 1, '%.2f'%(1e3*mu_c),style) 
sheet1.write(8, 2, '%.2f'%(1e3*mu_f),style) 

sheet1.write(9, 0, 'Calor específico J/Kg C')
sheet1.write(9, 1, '%.1f'%C_c)
sheet1.write(9, 2, '%.1f'%C_f)

sheet1.write(10, 0, 'Conductividad térmica W/Kg C',style) 
sheet1.write(10, 1, '%.3f'%(k_c),style) 
sheet1.write(10, 2, '%.3f'%(k_f),style) 

sheet1.write(11, 0, 'Presión de entrada kPa') 
sheet1.write(11, 1, '%.1f'%(101.3))
sheet1.write(11, 2, '%.1f'%(101.3))

sheet1.write(13, 0, 'Caída de presión Pa') 
sheet1.write(13, 1, '%.3f'%(DeltaP_tubos))
sheet1.write(13, 2, '%.3f'%(DeltaP_carcasa))

sheet1.write(14, 0, 'Resistencia de ensuciamiento m2C/W',style) 
sheet1.write(14, 1, '%.4f'%(Rf),style)
sheet1.write(14, 2, '%.4f'%(Rf),style)

sheet1.write(15, 0, 'Calor intercambiado kW') 
sheet1.write(15, 1, '%.1f'%(Q_old/1e3))
sheet1.write(15, 2, '-----')

sheet1.write(16, 0, 'Coeficiente Global de Transferencia',style)
sheet1.write(16, 1, '%.1f'%(U),style)
sheet1.write(16, 2, '-----',style)

sheet1.write(17, 0, 'Número de pasos de tubos')
sheet1.write(17, 1, '%0d'%Npt)
sheet1.write(17, 2, '-----')

sheet1.write(18, 0, 'Número de pasos de carcasa',style)
sheet1.write(18, 1, '%.1f'%(Nps),style)
sheet1.write(18, 2, '-----',style)

sheet1.write(19, 0, 'Tubos Longitud m')
sheet1.write(19, 1, '%.2f'%Lt)
sheet1.write(19, 2, '-----')

sheet1.write(20, 0, 'Tubos D exterior mm',style)
sheet1.write(20, 1, '%.2f'%(D*1e3),style)
sheet1.write(20, 2, '-----',style)

sheet1.write(21, 0, 'Tubos D interior mm')
sheet1.write(21, 1, '%.2f'%(Di*1e3))
sheet1.write(21, 2, '-----')
             
sheet1.write(22, 0, 'Tubos espesor mm',style)
sheet1.write(22, 1, '%.2f'%(e*1e3),style)
sheet1.write(22, 2, '-----')
             
sheet1.write(23,0,'Diámetro de la carcasa mm')             
sheet1.write(23,1,'%.2f'%(Ds*1e3))
sheet1.write(23,2,'-----')

nombre_salida = 'intercambiador_casco_y_tubos.xls'
wb.save(nombre_salida)
data_salida_excel = pd.read_excel('./'+nombre_salida,skiprows=2)
data_salida_excel

,Unnamed: 0,Unnamed: 1,Unnamed: 2
0,Nombre Fluido,Agua,Etilenglicol
1,Caudal masico (kg/s),6.68,4.50
2,Temperatura Entrada,65.0,5.0
3,Temperatura salida,43.0,40.0
4,Densidad kg/m3,986.25,1023.34
5,Viscosidad cP,0.51,1.56
6,Calor específico J/Kg C,4182.6,3900.5
7,Conductividad térmica W/Kg C,0.645,0.510
8,Presión de entrada kPa,101.3,101.3
9,NaN,NaN,NaN
